# Create Storm Events Database File
Clean data, converting to Parquet format and create a DuckDB database file.  

Author: Mark Bauer

In [1]:
import os
import ibis
import duckdb

ibis.options.interactive = True

In [2]:
%reload_ext watermark
%watermark -v -p duckdb,ibis

Python implementation: CPython
Python version       : 3.8.13
IPython version      : 8.4.0

duckdb: 0.10.0
ibis  : 3.2.0



In [3]:
# preview file names
!ls data/raw/ | head -n 5

StormEvents_details-ftp_v1.0_d1950_c20210803.csv.gz
StormEvents_details-ftp_v1.0_d1951_c20210803.csv.gz
StormEvents_details-ftp_v1.0_d1952_c20210803.csv.gz
StormEvents_details-ftp_v1.0_d1953_c20210803.csv.gz
StormEvents_details-ftp_v1.0_d1954_c20210803.csv.gz


In [4]:
# create a DuckDB database instance
con = duckdb.connect()

# create table of the storm_events data, glob files together
# take note on all_varchar and ignore_errors parameters
con.sql(
    """
    CREATE TABLE storm_events AS
    FROM read_csv('data/raw/*.csv.gz', all_varchar=True, ignore_errors=True)
    """
)

# examine count of rows
con.sql("SELECT COUNT(*) AS count_rows FROM storm_events").show()

┌────────────┐
│ count_rows │
│   int64    │
├────────────┤
│    1870438 │
└────────────┘



In [5]:
# examine column metadata and assign as a pandas df
describe_df = con.sql("DESCRIBE storm_events").df()

describe_df

,column_name,column_type,null,key,default,extra
0,BEGIN_YEARMONTH,VARCHAR,YES,None,None,None
1,BEGIN_DAY,VARCHAR,YES,None,None,None
2,BEGIN_TIME,VARCHAR,YES,None,None,None
3,END_YEARMONTH,VARCHAR,YES,None,None,None
4,END_DAY,VARCHAR,YES,None,None,None
5,END_TIME,VARCHAR,YES,None,None,None
6,EPISODE_ID,VARCHAR,YES,None,None,None
7,EVENT_ID,VARCHAR,YES,None,None,None
8,STATE,VARCHAR,YES,None,None,None
9,STATE_FIPS,VARCHAR,YES,None,None,None


In [6]:
# calculate summary statistics of each column
summarize_df = con.sql("SUMMARIZE storm_events").df()

summarize_df

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,BEGIN_YEARMONTH,VARCHAR,195001,202402,893,NaN,NaN,NaN,NaN,NaN,1870438,0.00
1,BEGIN_DAY,VARCHAR,1,9,31,NaN,NaN,NaN,NaN,NaN,1870438,0.00
2,BEGIN_TIME,VARCHAR,0,959,1413,NaN,NaN,NaN,NaN,NaN,1870438,0.00
3,END_YEARMONTH,VARCHAR,195001,202402,893,NaN,NaN,NaN,NaN,NaN,1870438,0.00
4,END_DAY,VARCHAR,1,9,31,NaN,NaN,NaN,NaN,NaN,1870438,0.00
5,END_TIME,VARCHAR,0,959,1413,NaN,NaN,NaN,NaN,NaN,1870438,0.00
6,EPISODE_ID,VARCHAR,1,99999,371735,NaN,NaN,NaN,NaN,NaN,1870438,12.42
7,EVENT_ID,VARCHAR,10,9999999,1835169,NaN,NaN,NaN,NaN,NaN,1870438,0.00
8,STATE,VARCHAR,ALABAMA,WYOMING,70,NaN,NaN,NaN,NaN,NaN,1870438,0.00
9,STATE_FIPS,VARCHAR,1,99,70,NaN,NaN,NaN,NaN,NaN,1870438,0.00


In [7]:
# preview data
sample_df = con.sql("SELECT * FROM storm_events LIMIT 20").df()

sample_df

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,195004,28,1445,195004,28,1445,None,10096222,OKLAHOMA,40,...,0,None,None,35.12,-99.20,35.17,-99.20,None,None,PUB
1,195004,29,1530,195004,29,1530,None,10120412,TEXAS,48,...,0,None,None,31.90,-98.60,31.73,-98.60,None,None,PUB
2,195007,5,1800,195007,5,1800,None,10104927,PENNSYLVANIA,42,...,0,None,None,40.58,-75.70,40.65,-75.47,None,None,PUB
3,195007,5,1830,195007,5,1830,None,10104928,PENNSYLVANIA,42,...,0,None,None,40.60,-76.75,None,None,None,None,PUB
4,195007,24,1440,195007,24,1440,None,10104929,PENNSYLVANIA,42,...,0,None,None,41.63,-79.68,None,None,None,None,PUB
5,195008,29,1600,195008,29,1600,None,10104930,PENNSYLVANIA,42,...,0,None,None,40.22,-75.00,None,None,None,None,PUB
6,195011,4,1700,195011,4,1700,None,10104931,PENNSYLVANIA,42,...,0,None,None,40.20,-76.12,40.27,-76.07,None,None,PUB
7,195011,4,1730,195011,4,1730,None,10104932,PENNSYLVANIA,42,...,0,None,None,40.27,-76.07,40.40,-75.93,None,None,PUB
8,195009,15,1745,195009,15,1745,None,10099490,OKLAHOMA,40,...,0,None,None,35.00,-96.25,35.07,-96.17,None,None,PUB
9,195009,16,130,195009,16,130,None,10099491,OKLAHOMA,40,...,0,None,None,34.83,-95.85,None,None,None,None,PUB


In [8]:
# there are too many columns to fit on the screen, so page through them 15 at a time
sample_df.iloc[:, :15]

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,YEAR,MONTH_NAME,EVENT_TYPE,CZ_TYPE,CZ_FIPS
0,195004,28,1445,195004,28,1445,None,10096222,OKLAHOMA,40,1950,April,Tornado,C,149
1,195004,29,1530,195004,29,1530,None,10120412,TEXAS,48,1950,April,Tornado,C,93
2,195007,5,1800,195007,5,1800,None,10104927,PENNSYLVANIA,42,1950,July,Tornado,C,77
3,195007,5,1830,195007,5,1830,None,10104928,PENNSYLVANIA,42,1950,July,Tornado,C,43
4,195007,24,1440,195007,24,1440,None,10104929,PENNSYLVANIA,42,1950,July,Tornado,C,39
5,195008,29,1600,195008,29,1600,None,10104930,PENNSYLVANIA,42,1950,August,Tornado,C,17
6,195011,4,1700,195011,4,1700,None,10104931,PENNSYLVANIA,42,1950,November,Tornado,C,71
7,195011,4,1730,195011,4,1730,None,10104932,PENNSYLVANIA,42,1950,November,Tornado,C,11
8,195009,15,1745,195009,15,1745,None,10099490,OKLAHOMA,40,1950,September,Tornado,C,63
9,195009,16,130,195009,16,130,None,10099491,OKLAHOMA,40,1950,September,Tornado,C,121


In [9]:
sample_df.iloc[:, 15:30]

,CZ_NAME,WFO,BEGIN_DATE_TIME,CZ_TIMEZONE,END_DATE_TIME,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,SOURCE,MAGNITUDE,MAGNITUDE_TYPE,FLOOD_CAUSE
0,WASHITA,None,28-APR-50 14:45:00,CST,28-APR-50 14:45:00,0,0,0,0,250K,0,None,0,None,None
1,COMANCHE,None,29-APR-50 15:30:00,CST,29-APR-50 15:30:00,0,0,0,0,25K,0,None,0,None,None
2,LEHIGH,None,05-JUL-50 18:00:00,CST,05-JUL-50 18:00:00,2,0,0,0,25K,0,None,0,None,None
3,DAUPHIN,None,05-JUL-50 18:30:00,CST,05-JUL-50 18:30:00,0,0,0,0,2.5K,0,None,0,None,None
4,CRAWFORD,None,24-JUL-50 14:40:00,CST,24-JUL-50 14:40:00,0,0,0,0,2.5K,0,None,0,None,None
5,BUCKS,None,29-AUG-50 16:00:00,CST,29-AUG-50 16:00:00,0,0,0,0,2.5K,0,None,0,None,None
6,LANCASTER,None,04-NOV-50 17:00:00,CST,04-NOV-50 17:00:00,1,0,0,0,250K,0,None,0,None,None
7,BERKS,None,04-NOV-50 17:30:00,CST,04-NOV-50 17:30:00,0,0,0,0,250K,0,None,0,None,None
8,HUGHES,None,15-SEP-50 17:45:00,CST,15-SEP-50 17:45:00,6,0,0,0,250K,0,None,0,None,None
9,PITTSBURG,None,16-SEP-50 01:30:00,CST,16-SEP-50 01:30:00,0,0,0,0,.25K,0,None,0,None,None


In [10]:
sample_df.iloc[:, 30:45]

,CATEGORY,TOR_F_SCALE,TOR_LENGTH,TOR_WIDTH,TOR_OTHER_WFO,TOR_OTHER_CZ_STATE,TOR_OTHER_CZ_FIPS,TOR_OTHER_CZ_NAME,BEGIN_RANGE,BEGIN_AZIMUTH,BEGIN_LOCATION,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT
0,None,F3,3.4,400,None,None,None,None,0,None,None,0,None,None,35.12
1,None,F1,11.5,200,None,None,None,None,0,None,None,0,None,None,31.90
2,None,F2,12.9,33,None,None,None,None,0,None,None,0,None,None,40.58
3,None,F2,0,13,None,None,None,None,0,None,None,0,None,None,40.60
4,None,F0,0,33,None,None,None,None,0,None,None,0,None,None,41.63
5,None,F1,1,33,None,None,None,None,0,None,None,0,None,None,40.22
6,None,F3,4.7,100,None,None,None,None,0,None,None,0,None,None,40.20
7,None,F3,11.2,100,None,None,None,None,0,None,None,0,None,None,40.27
8,None,F2,6.8,100,None,None,None,None,0,None,None,0,None,None,35.00
9,None,F0,0,33,None,None,None,None,0,None,None,0,None,None,34.83


In [11]:
sample_df.iloc[:, 45:]

,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,-99.20,35.17,-99.20,None,None,PUB
1,-98.60,31.73,-98.60,None,None,PUB
2,-75.70,40.65,-75.47,None,None,PUB
3,-76.75,None,None,None,None,PUB
4,-79.68,None,None,None,None,PUB
5,-75.00,None,None,None,None,PUB
6,-76.12,40.27,-76.07,None,None,PUB
7,-76.07,40.40,-75.93,None,None,PUB
8,-96.25,35.07,-96.17,None,None,PUB
9,-95.85,None,None,None,None,PUB


In [12]:
# calculate summary statistics of each column again
summarize_df = con.sql("SUMMARIZE storm_events").df()

summarize_df

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,BEGIN_YEARMONTH,VARCHAR,195001,202402,893,NaN,NaN,NaN,NaN,NaN,1870438,0.00
1,BEGIN_DAY,VARCHAR,1,9,31,NaN,NaN,NaN,NaN,NaN,1870438,0.00
2,BEGIN_TIME,VARCHAR,0,959,1413,NaN,NaN,NaN,NaN,NaN,1870438,0.00
3,END_YEARMONTH,VARCHAR,195001,202402,893,NaN,NaN,NaN,NaN,NaN,1870438,0.00
4,END_DAY,VARCHAR,1,9,31,NaN,NaN,NaN,NaN,NaN,1870438,0.00
5,END_TIME,VARCHAR,0,959,1413,NaN,NaN,NaN,NaN,NaN,1870438,0.00
6,EPISODE_ID,VARCHAR,1,99999,371735,NaN,NaN,NaN,NaN,NaN,1870438,12.42
7,EVENT_ID,VARCHAR,10,9999999,1835169,NaN,NaN,NaN,NaN,NaN,1870438,0.00
8,STATE,VARCHAR,ALABAMA,WYOMING,70,NaN,NaN,NaN,NaN,NaN,1870438,0.00
9,STATE_FIPS,VARCHAR,1,99,70,NaN,NaN,NaN,NaN,NaN,1870438,0.00


In [13]:
# we want to cast columns to their correct dtype, select only desired columns, clean up the dollar values
# and finally save file as Parquet
# notice we are only interested in records where property damage not null

query = """
    SELECT
        CAST(EVENT_ID AS INTEGER) AS EVENT_ID,
        STATE,
        CAST(STATE_FIPS AS INTEGER) AS STATE_FIPS,
        CAST(YEAR AS INTEGER) AS YEAR,
        MONTH_NAME,
        EVENT_TYPE,
        CZ_TYPE,
        CAST(CZ_FIPS AS INTEGER) AS CZ_FIPS,
        CZ_NAME,
        WFO,
        BEGIN_DATE_TIME,
        END_DATE_TIME,
        CAST(INJURIES_DIRECT AS INTEGER) AS INJURIES_DIRECT,
        CAST(INJURIES_INDIRECT AS INTEGER) AS INJURIES_INDIRECT,
        CAST(DEATHS_DIRECT AS INTEGER) AS DEATHS_DIRECT,
        CAST(DEATHS_INDIRECT AS INTEGER) AS DEATHS_INDIRECT,
        DAMAGE_PROPERTY,
        DAMAGE_CROPS,
        SOURCE,
        CASE 
            WHEN RIGHT(DAMAGE_PROPERTY, 1) = 'K' AND LEN(DAMAGE_PROPERTY) > 1
                THEN CAST(SUBSTRING(DAMAGE_PROPERTY, 1, LENGTH(DAMAGE_PROPERTY) - 1) AS DOUBLE) * 1000
            WHEN RIGHT(DAMAGE_PROPERTY, 1) = 'M' AND LEN(DAMAGE_PROPERTY) > 1
                THEN CAST(SUBSTRING(DAMAGE_PROPERTY, 1, LENGTH(DAMAGE_PROPERTY) - 1) AS DOUBLE) * 1000000
            ELSE 0
            END AS DAMAGE_PROPERTY_CONVERTED

    FROM
        storm_events
    WHERE
        DAMAGE_PROPERTY NOT NULL
"""

con.sql(query).write_parquet("data/storm-events.parquet") 

In [14]:
ls data/

raw/                  storm-events.parquet


In [15]:
# sanity check our data

# create a DuckDB database instance
con = duckdb.connect()

# create table of the claims dataset
con.sql(
    """
    CREATE TABLE storm_events AS
    FROM read_parquet('data/storm-events.parquet')
    """
)

# examine count of rows
con.sql("SELECT COUNT(*) AS count_rows FROM storm_events").show()

┌────────────┐
│ count_rows │
│   int64    │
├────────────┤
│    1295193 │
└────────────┘



In [16]:
# sanity check out damage property columns
summarize_df = (
    con
    .sql("SUMMARIZE SELECT EVENT_TYPE, DAMAGE_PROPERTY, DAMAGE_PROPERTY_CONVERTED FROM storm_events")
    .df()
)

summarize_df

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,EVENT_TYPE,VARCHAR,Astronomical Low Tide,Winter Weather,70,None,None,None,None,None,1295193,0.0
1,DAMAGE_PROPERTY,VARCHAR,.01K,K,3091,None,None,None,None,None,1295193,0.0
2,DAMAGE_PROPERTY_CONVERTED,DOUBLE,0.0,950000000.0,2164,222760.43361035068,7243742.210315917,0.0,0.0,2007.3849283328711,1295193,0.0


In [17]:
# sanity check out damage property columns and assign to damage_df
damage_df = (
    con
    .sql("SELECT EVENT_TYPE, DAMAGE_PROPERTY, DAMAGE_PROPERTY_CONVERTED FROM storm_events")
    .df()
)

damage_df

,EVENT_TYPE,DAMAGE_PROPERTY,DAMAGE_PROPERTY_CONVERTED
0,Tornado,250K,250000.0
1,Tornado,25K,25000.0
2,Tornado,25K,25000.0
3,Tornado,2.5K,2500.0
4,Tornado,2.5K,2500.0
...,...,...,...
1295188,Hail,0.00K,0.0
1295189,Hail,0.00K,0.0
1295190,Hail,0.00K,0.0
1295191,Thunderstorm Wind,25.00K,25000.0


In [18]:
# sanity check out damage property columns
damage_df.loc[damage_df['DAMAGE_PROPERTY'].isnull()]

,EVENT_TYPE,DAMAGE_PROPERTY,DAMAGE_PROPERTY_CONVERTED


In [19]:
# preview smallest damage property values
damage_df.sort_values(by=['DAMAGE_PROPERTY'])

,EVENT_TYPE,DAMAGE_PROPERTY,DAMAGE_PROPERTY_CONVERTED
316976,Hail,.01K,10.0
321573,Thunderstorm Wind,.01K,10.0
273108,Hail,.01K,10.0
235306,Hail,.01K,10.0
336179,Hail,.01K,10.0
...,...,...,...
341376,High Wind,K,0.0
341377,High Wind,K,0.0
341404,High Wind,K,0.0
341436,High Wind,K,0.0


In [20]:
# preview smallest damage property values where value != K
# note I am treating this as $0 for now
damage_df.loc[damage_df['DAMAGE_PROPERTY'] != 'K'].sort_values(by=['DAMAGE_PROPERTY'])

,EVENT_TYPE,DAMAGE_PROPERTY,DAMAGE_PROPERTY_CONVERTED
303250,Thunderstorm Wind,.01K,10.0
239348,Thunderstorm Wind,.01K,10.0
239321,Hail,.01K,10.0
239320,Hail,.01K,10.0
305166,Thunderstorm Wind,.01K,10.0
...,...,...,...
355711,High Wind,9M,9000000.0
240596,Flood,9M,9000000.0
327280,Hail,9M,9000000.0
241068,Tornado,9M,9000000.0


In [21]:
# preview largest damage property values
damage_df.sort_values(by=['DAMAGE_PROPERTY_CONVERTED'], ascending=False).head(20)

,EVENT_TYPE,DAMAGE_PROPERTY,DAMAGE_PROPERTY_CONVERTED
1069001,High Wind,950.00M,950000000.0
742882,Tornado,910.00M,910000000.0
1056892,Hurricane,900.00M,900000000.0
608745,Hail,900.00M,900000000.0
701602,Hail,900.00M,900000000.0
357187,High Wind,881M,881000000.0
456486,Hurricane (Typhoon),850M,850000000.0
456495,Hurricane (Typhoon),850M,850000000.0
1057051,Hurricane,800.00M,800000000.0
926462,Flash Flood,750.00M,750000000.0


In [22]:
# create a DuckDB database instance
con = duckdb.connect("data/storm-events.db")

# create table of the claims dataset
con.sql(
    """
    CREATE TABLE storm_events AS
    FROM read_parquet('data/storm-events.parquet')
    """
)

# examine count of rows
con.sql("SELECT COUNT(*) AS count_rows FROM storm_events").show()

┌────────────┐
│ count_rows │
│   int64    │
├────────────┤
│    1295193 │
└────────────┘



In [23]:
con.close()